In [1]:
from sklearn.ensemble import RandomForestClassifier
import numpy as np
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from matplotlib.cm import tab20 as cmap
from matplotlib import pyplot as plt
import laspy as lp
from sklearn.metrics import confusion_matrix

# Load the train and test data for the NN

In [14]:
X_train = np.load('test_data_ahn.npy')
y_train = np.load('targets_ahn_test.npy')
X_test = np.load('test_data_ahn.npy')
y_test = np.load('targets_ahn_test.npy')

In [15]:
X_test_features = x_new_test2[:,3:]
test_coordinates = X_test[:,:3]
X_train = X_train[:,3:]

In [16]:
# Create a scaler object
scaler = StandardScaler()

# Fit the scaler to the training data and transform the data
X_train_scaled = scaler.fit_transform(X_train)

X_test_scaled = scaler.fit_transform(X_test_features)

# First version of NN

In [ ]:
# Define your Neural Network architecture
model = Sequential()
model.add(Dense(128, activation='relu', input_dim=8))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(32, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(16, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(7, activation='softmax')) # 7 is the number of classes in your target variable

# Compile your Neural Network
optimizer = Adam(learning_rate=0.001)
model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# Train your Neural Network on the same validation set
history = model.fit(X_train_scaled, y_train, epochs=8, batch_size=32)

# Evaluate your Neural Network on the testing set
loss, accuracy = model.evaluate(X_test_scaled, y_test)

# Print the accuracy as a percentage
print("Accuracy: {:.2f}%".format(accuracy * 100))


In [ ]:
# Predict data
# Use your trained Neural Network to predict probabilities on new data
nn_probs = model.predict(X_test_scaled)

# Convert the probabilities to class labels
nn_labels = np.argmax(nn_probs, axis=-1)

In [ ]:
y_pred_modified = np.argmax(model.predict(X_test_scaled), axis=-1)
cm_modified = confusion_matrix(y_test, y_pred_modified)

class_accuracies_modified = cm_modified.diagonal() / cm_modified.sum(axis=1)
for i, acc in enumerate(class_accuracies_modified):
    print(f"Accuracy for class {i} (Modified architecture): {acc:.2f}")

# Another version of NN

In [ ]:
from tensorflow.keras.layers import LeakyReLU, PReLU
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau

# Define your Neural Network architecture
model = Sequential()
model.add(Dense(256, input_dim=8))
model.add(LeakyReLU(alpha=0.1))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(128))
model.add(PReLU())
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(32, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(7, activation='softmax'))

# Compile your Neural Network
optimizer = Adam(learning_rate=0.001)
model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# Set up a learning rate scheduler
#reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr=1e-6)

# Train your Neural Network on the same validation set
history = model.fit(X_train_scaled, y_train, epochs=8, batch_size=64) #validation_split=0.2, callbacks=[reduce_lr])

# Evaluate your Neural Network on the testing set
loss, accuracy = model.evaluate(X_test_scaled, y_test)

# Print the accuracy as a percentage
print("Accuracy: {:.2f}%".format(accuracy * 100))


In [ ]:
# Evaluate your Neural Network on the testing set
loss, accuracy = model.evaluate(X_test_scaled, y_test)

# Print the accuracy as a percentage
print("Accuracy: {:.2f}%".format(accuracy * 100))

In [ ]:
y_pred_modified = np.argmax(model.predict(X_test_scaled), axis=-1)
cm_modified = confusion_matrix(y_test, y_pred_modified)

class_accuracies_modified = cm_modified.diagonal() / cm_modified.sum(axis=1)
for i, acc in enumerate(class_accuracies_modified):
    print(f"Accuracy for class {i} (Modified architecture): {acc:.2f}")


In [ ]:
# Use your trained Neural Network to predict probabilities on new data
nn_probs = model.predict(X_test_scaled)

# Convert the probabilities to class labels
nn_labels = np.argmax(nn_probs, axis=-1)

# Save the predicted area as an las file

In [ ]:
print_coord = x_new_test2[:, :3]
fn = "Final_of_day_pred_preds.las"

vals = np.linspace(0, 1, 100)
np.random.shuffle(vals)
cmap = plt.cm.colors.ListedColormap(plt.cm.tab20(vals))
header = lp.header.Header()
header.data_format_id = 2
fp = lp.file.File(fn, mode = 'w', header = header)
fp.header.scale = [0.01, 0.01, 0.01]
fp.header.offset = [min(print_coord[:,0]), min(print_coord[:,1]), min(print_coord[:,2])]
fp.x = print_coord[:, 0]
fp.y = print_coord[:, 1]
fp.z = print_coord[:, 2]
fp.pt_src_id = nn_labels
#fp.intensity = intensity
fp.close()